In [ ]:
import torch

'''
1. 设置随机数种子
'''

In [ ]:
# 设置随机数种子
import numpy as np
import random
seed = 42
random.seed(seed)
os.environ["PYTHONHASHSEED"] = str(seed)
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed(seed) # 为当前的GPU设置产生随机数的种子
# torch.cuda.manual_seed_all(seed) # 为所有GPU设置产生随机数的种子
torch.backends.cudnn.deterministic = True # 每次返回的卷积算法将是确定的; 会降低训练速度; 从checkpoints重新开始时会出现意外的结果
# torch.backends.cudnn.benchmark = False # 设置为True, 可以大大提升卷积神经网络的运行速度; 可在网络训练开始前设置